# Import lib

In [1]:
import pickle
import numpy as np
import string
import re
from pyvi import ViTokenizer
import matplotlib.pyplot as plt
import nltk
import asyncio

from tkinter import *




pattern = r'[' + string.punctuation +']'
#from nltk.tokenize import word_tokenize

In [2]:
#sentences = ("Trong khi đó, trong kỳ họp Quốc hội bất thường khai mạc ngày 4/1, Chính phủ có trình Quốc hội dự thảo một luật sửa 8 luật (trong đó có Luật Đầu tư), nhưng lại không có nội dung về sửa đổi phân cấp, phân quyền chấp thuận chủ trương đầu tư xây dựng khu công nghiệp. 'Chúng tôi đề nghị Chính phủ bổ sung vấn đề này vào để sớm sửa quy định. Đi theo với sự phân cấp, phân quyền là tăng cường kiểm tra, giám sát, cá thể hóa trách nhiệm', ông Quảng đề xuất. Được mời phát biểu, Chủ tịch UBND TP.HCM Phan Văn Mãi cũng nhắc đến một luật sửa 8 luật đang được Chính phủ trình Quốc hội. Ông đề nghị sau khi Quốc hội thông qua thì Chính phủ sớm có hướng dẫn, phân cấp, phân quyền để luật đi vào cuộc sống. Những vấn đề cần phân cấp, phân quyền được Chủ tịch UBND TP.HCM đề xuất là phê chuẩn, điều chỉnh cục bộ quy hoạch, tách dự án giải phóng mặt bằng ra khỏi dự án thông thường... Lãnh đạo TP.HCM cũng đề nghị, kiến nghị sớm sửa đổi Luật Đất đai. Là tham luận cuối cùng trong buổi sáng, Bí thư Thành ủy Thái Nguyên Nguyễn Thanh Hải cũng đề nghị quan tâm phân cấp, phân quyền nhiều hơn cho địa phương. Bà Hải đặc biệt nhấn mạnh đến việc triển khai tách dự án giải phóng mặt bằng ra khỏi dự án thông thường, như vậy mới có thể tạo sự đột biến trong việc thu hút đầu tư.")

In [3]:

def btn_clicked():
    scale_value = scale.get()
    entry1.delete('1.0', END)
    
    sentences = entry0.get("1.0", "end-1c")
    #print(sentences)
    if sentences == '':
        print('empty')
        entry1.insert(END, "Empty..")
    else:
        OUTPUT = "   " + summarize(sentences, scale_value)
    
        entry1.delete('1.0', END)
        entry1.insert(END, OUTPUT)
    
    
    
    

In [4]:
def summarize(sentences, scale_value):
    #Number of words in input text
    words_in = ViTokenizer.tokenize(sentences)
    words_in = re.sub(pattern, '', words_in)
    number_of_words_in = len(words_in.split())
    
    #split sentences - lowercase
    #sentences=sentences.lower().strip()
    sentences=sentences.strip()
    print(sentences)
    
    sentences = nltk.sent_tokenize(sentences)
    
    number_of_sentences = len(sentences)
    
    
    import sklearn.model_selection
    from gensim.models import KeyedVectors 
    w2v = KeyedVectors.load_word2vec_format("Vectors-sg/model.vec")
    
    vocab = w2v.vocab
    
    sentences_vec_array = []
    for sentence in sentences:
        sentence = re.sub(pattern, '', sentence)
        sentence = ViTokenizer.tokenize(sentence).strip()
        words = sentence.split()
        sentence_vec = np.zeros((300))
        for word in words:
            if word in vocab:
                sentence_vec+=w2v.wv[word]
        sentences_vec_array.append(sentence_vec)
        
    #cluster
    from sklearn.cluster import KMeans

    #n_clusters = 5
    n_clusters = round((number_of_sentences/8)*scale_value)
    kmeans = KMeans(n_clusters=n_clusters)
    kmeans = kmeans.fit(sentences_vec_array)
    
    #choose sentence for each cluster
    from sklearn.metrics import pairwise_distances_argmin_min

    avg = []
    for j in range(n_clusters):
        idx = np.where(kmeans.labels_ == j)[0]
        avg.append(np.mean(idx))

    closest, _ = pairwise_distances_argmin_min(kmeans.cluster_centers_, sentences_vec_array)
    ordering = sorted(range(n_clusters), key=lambda k: avg[k])
    summary = '\n\n   '.join([sentences[closest[idx]] for idx in ordering])
    summary_e = ' '.join([sentences[closest[idx]] for idx in ordering])
    
    #Number of words in final text
    words_out = ViTokenizer.tokenize(summary)
    number_of_words_out = len(words_out.split())
    
    
    #print("tom tat: ")
    #print(words_out)
    print(summary_e)
    print("Number_of_sentences: " + str(number_of_sentences))
    print("number_of_words_in: " + str(number_of_words_in))
    print("number_of_words_out: " + str(number_of_words_out))
    print("div: " + str((number_of_words_out/number_of_words_in)*100))
    print("number_of_cluster: " + str(n_clusters))
    return summary
    
    
    

In [5]:
if __name__ == "__main__":
    window = Tk()

    window.geometry("1005x571")
    window.configure(bg = "#ffffff")
    window.title("Auto Text Summarization")
    canvas = Canvas(
        window,
        bg = "#ffffff",
        height = 571,
        width = 1005,
        bd = 0,
        highlightthickness = 0,
        relief = "ridge")
    canvas.place(x = 0, y = 0)

    background_img = PhotoImage(file = f"GUI/background.png")
    background = canvas.create_image(
        496.5, 291.0,
        image=background_img)

    entry0_img = PhotoImage(file = f"GUI/img_textBox0.png")
    entry0_bg = canvas.create_image(
        253.5, 280.5,
        image = entry0_img)

    entry0 = Text(
        bd = 0,
        bg = "#feffff",
        highlightthickness = 0,
        wrap=WORD)

    entry0.place(
        x = 35.0, y = 56,
        width = 437.0,
        height = 457)

    entry1_img = PhotoImage(file = f"GUI/img_textBox1.png")
    entry1_bg = canvas.create_image(
        754.5, 280.5,
        image = entry1_img)

    entry1 = Text(
        bd = 0,
        bg = "#feffff",
        highlightthickness = 0,
        wrap=WORD)

    entry1.place(
        x = 536.0, y = 56,
        width = 437.0,
        height = 457)
    
    scale = Scale(
        window,
        from_=1,
        to=5,
        orient = 'horizontal',
        length = 150,)
    scale.place(x = 115, y = 5)
    
    label = Label(window, text = "Mức độ tóm tắt:")
    label.place(x = 25, y = 5)

    img0 = PhotoImage(file = f"GUI/img0.png")
    b0 = Button(
        image = img0,
        borderwidth = 0,
        highlightthickness = 0,
        command = lambda:btn_clicked(),
        relief = "flat")

    b0.place(
        x = 200, y = 525,
        width = 108,
        height = 39)
    
    window.resizable(False, False)
    window.mainloop()

Bàn ấn định chiến thắng của Quang Hải trước Thanh Hóa là bằng chứng cho thấy CLB Hà Nội chưa hề sẵn sàng cho cuộc chia ly với cầu thủ đẳng cấp nhất bóng đá Việt Nam hiện tại.
Gần đến ngày ra đi, Hải vẫn là nhân tổ cực kỳ quan trọng ở CLB Hà Nội. Anh là người ghi bàn duy nhất của trận đấu, là người mang về 3 điểm quý giá cho đội bóng trong bối cảnh phần còn lại bất lực trước đối thủ xứ Thanh. Thú vị hơn, anh làm điều đó chỉ sau 45 phút có mặt trên sân.Số 19 của CLB Hà Nội là cầu thủ hàng đầu Việt Nam hiện nay. Điều đó đã được thừa nhận trong thời gian dài vừa qua. Việc đưa ra những con số thống kê để khẳng định điều đó xem ra là thừa thãi. Ở CLB Hà Nội, Quang Hải, dù không phải thủ lĩnh, vẫn đóng vai trò quan trọng hơn cả trong cách vận hành chiến thuật.

Trước Thanh Hóa tối 16/3, CLB Hà Nội có và không có Quang Hải là hai đội bóng hoàn toàn khác nhau.

Hiệp một, tiền vệ sinh năm 1997 ngồi dự bị, CLB Hà Nội không thực sự áp đảo và có xu hướng chơi bóng dài nhiều hơn. Với đầy đủ các hảo 

C:\Users\lego2\anaconda3\envs\python36-clone-GUI\lib\site-packages\ipykernel_launcher.py:31: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).


Với đầy đủ các hảo thủ như Văn Quyết, Hùng Dũng, Tuấn Hải, họ vẫn bất lực trước hệ thống phòng ngự kỷ luật của Thanh Hóa. Anh cầm bóng, làm chủ tuyến giữa, tổ chuyền, chuyền bóng, chọc khe, tạo ra một thế trận sáng sủa hơn hẳn cho đội của mình. Trước câu hỏi của Zing liệu ông và cộng sự có tự tin với lực lượng hiện tại của CLB Hà Nội nếu thiếu vắng Quang Hải, nhà cầm quân người Hàn Quốc không trả lời trực diện. Cũng chính Hải là người mang về cho đội nhà quả phạt góc ở phút 81 và từ quả phạt góc đó, anh ghi bàn thắng duy nhất của trận đấu. Trước Thanh Hóa tối 16/3, CLB Hà Nội có và không có Quang Hải là hai đội bóng hoàn toàn khác nhau. Anh là người ghi bàn duy nhất của trận đấu, là người mang về 3 điểm quý giá cho đội bóng trong bối cảnh phần còn lại bất lực trước đối thủ xứ Thanh. Thú vị hơn, anh làm điều đó chỉ sau 45 phút có mặt trên sân.Số 19 của CLB Hà Nội là cầu thủ hàng đầu Việt Nam hiện nay. Giá trị của Quang Hải không dừng lại ở chuyên môn. Ông Petrovic bên phía Thanh Hóa cho

C:\Users\lego2\anaconda3\envs\python36-clone-GUI\lib\site-packages\ipykernel_launcher.py:31: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).


Bộ trưởng Tài chính Nga Anton Siluanov cho biết khả năng Moscow thanh toán nợ trái phiếu bằng đồng USD đang bị bỏ ngỏ và phụ thuộc vào Mỹ. Tuy nhiên, ông cho biết Mỹ cần làm rõ liệu Nga có thể tiếp cận các tài khoản ngoại tệ để thanh toán hay không.Sau khi đưa quân vào Ukraine, Nga đối mặt một loạt biện pháp trừng phạt đánh vào kinh tế của phương Tây cũng như bị loại khỏi hệ thống tài chính toàn cầu. Đồng thời, các trái chủ từ quốc gia “thù địch” có thể sẽ được thanh toán bằng RUB. Việc không thanh toán sẽ đánh dấu vụ vỡ nợ có chủ quyền đầu tiên của Nga kể từ năm 1998 đối với nợ nội bộ và lần đầu tiên đối với nợ ngoại tệ kể từ cuộc Cách mạng Bolshevik năm 1918.
Number_of_sentences: 15
number_of_words_in: 328
number_of_words_out: 121
div: 36.890243902439025
number_of_cluster: 4
Ukraine đang đề xuất trở thành một quốc gia phi quân sự có lực lượng vũ trang riêng giống như Áo và Thụy Điển, trưởng đoàn đàm phán của Nga Vladimir Medinsky cho biết hôm 16/3, TASS đưa tin.

Một quy chế tương tự

C:\Users\lego2\anaconda3\envs\python36-clone-GUI\lib\site-packages\ipykernel_launcher.py:31: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).


Reuters dẫn lời ông Mykhailo Podoliak - trưởng đoàn đàm phán Kyiv, cố vấn Tổng thống Ukraine Volodymyr Zelensky - bác bỏ mô hình Áo hay Thụy Điển, nói rằng bất kỳ thỏa thuận nào về mô hình Ukraine đều cần đảm bảo an ninh phòng trường hợp Ukraine bị tấn công.Sau Thế chiến II, lãnh thổ Áo nói chung và thủ đô Vienna nói riêng bị quân Đồng minh chiếm giữ và chia làm bốn phần do Liên Xô, Mỹ, Anh Pháp kiểm soát. Cùng năm, nghị viện Áo thông qua luật hiến pháp về tình trạng trung lập

Hiệp ước năm 1955, dù cho phép Áo có lực lượng vũ trang riêng, đề ra nguyên tắc ba không của Áo: Không tham gia liên minh quân sự; không cho phép quân đội nước ngoài đặt căn cứ quân sự trên lãnh thổ; không tham gia chiến tranh. Tình trạng trung lập của Áo cũng giúp nước này được ghi nhận trên trường quốc tế, với việc là chủ nhà của những hội nghị thượng đỉnh giữa các lãnh đạo Mỹ và Liên Xô trong Chiến tranh Lạnh. Việc Áo không tham gia NATO là chủ đề được thảo luận trong nhiều thập niên ở nước này. Trong khi đó,

C:\Users\lego2\anaconda3\envs\python36-clone-GUI\lib\site-packages\ipykernel_launcher.py:31: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).


Reuters dẫn lời ông Mykhailo Podoliak - trưởng đoàn đàm phán Kyiv, cố vấn Tổng thống Ukraine Volodymyr Zelensky - bác bỏ mô hình Áo hay Thụy Điển, nói rằng bất kỳ thỏa thuận nào về mô hình Ukraine đều cần đảm bảo an ninh phòng trường hợp Ukraine bị tấn công.Sau Thế chiến II, lãnh thổ Áo nói chung và thủ đô Vienna nói riêng bị quân Đồng minh chiếm giữ và chia làm bốn phần do Liên Xô, Mỹ, Anh Pháp kiểm soát. Cho đến nay, Hiệp ước Nhà nước Áo vẫn là một phần của hiến pháp nước này, và ảnh hưởng đến lập trường của Áo trong các vấn đề quốc tế.Lần gần nhất Thụy Điển tham gia một cuộc chiến đã từ năm 1814, trong chiến tranh Napoleon. Việc Áo không tham gia NATO là chủ đề được thảo luận trong nhiều thập niên ở nước này. Trong khi đó, các quan điểm đối lập cho biết việc tham gia NATO sẽ tăng cường an ninh cho Stockholm theo Điều 5 NATO - nhấn mạnh vào đảm bảo an ninh tập thể. Quá trình để một quốc gia tham gia NATO vốn kéo dài nhiều năm, nhưng Tổng thư ký NATO Jens Stoltenberg cho biết có thể r